In [16]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable


In [17]:
pip install caer

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.5/809.5 KB 297.9 kB/s eta 0:00:000:00:01m eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 677.3 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.3/68.3 MB 730.8 kB/s eta 0:00:00m eta 0:00:010:00:03
Note: you may need to restart the kernel to use updated packages.


In [18]:
!pip install caer canaro

Defaulting to user installation because normal site-packages is not writeable


In [81]:
import matplotlib.pyplot as plt
import os
import gc

import cv2 as cv
import caer
import canaro
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

In [27]:
img_size=(180,180)
channels = 1
char_path= 'Data/simpsons_dataset'

In [28]:
char_dict={}
for char in os.listdir(char_path):
    char_dict[char]=len(os.listdir(os.path.join(char_path,char)))
char_dict = caer.sort_dict(char_dict, descending=True)
char_dict

[('homer_simpson', 2246),
 ('ned_flanders', 1454),
 ('moe_szyslak', 1452),
 ('lisa_simpson', 1354),
 ('bart_simpson', 1342),
 ('marge_simpson', 1291),
 ('krusty_the_clown', 1206),
 ('principal_skinner', 1194),
 ('charles_montgomery_burns', 1193),
 ('milhouse_van_houten', 1079),
 ('chief_wiggum', 986),
 ('abraham_grampa_simpson', 913),
 ('sideshow_bob', 877),
 ('apu_nahasapeemapetilon', 623),
 ('kent_brockman', 498),
 ('comic_book_guy', 469),
 ('edna_krabappel', 457),
 ('nelson_muntz', 358),
 ('lenny_leonard', 310),
 ('mayor_quimby', 246),
 ('waylon_smithers', 181),
 ('maggie_simpson', 128),
 ('groundskeeper_willie', 121),
 ('barney_gumble', 106),
 ('selma_bouvier', 103),
 ('carl_carlson', 98),
 ('ralph_wiggum', 89),
 ('patty_bouvier', 72),
 ('martin_prince', 71),
 ('professor_john_frink', 65),
 ('snake_jailbird', 55),
 ('cletus_spuckler', 47),
 ('rainier_wolfcastle', 45),
 ('simpsons_dataset', 42),
 ('agnes_skinner', 42),
 ('sideshow_mel', 40),
 ('otto_mann', 32),
 ('gil', 27),
 ('fat_

In [31]:
chars = []
cnt=0
for i in char_dict:
    chars.append(i[0])
    cnt+=1
    if cnt>=3:
        break
chars

['homer_simpson', 'ned_flanders', 'moe_szyslak']

In [32]:
#Создание тренировочных данных
train=caer.preprocess_from_dir(char_path, chars, channels=channels, IMG_SIZE=img_size, isShuffle=True)

[INFO] Could not find a file to load from. Generating the training data
----------------------------------------------
[INFO] At 1000 files
[INFO] At 2000 files
[INFO] 2246 files found in 0.00845789909362793s
[INFO] At 1000 files
[INFO] 1454 files found in 0.0031936168670654297s
[INFO] At 1000 files
[INFO] 1452 files found in 0.003599882125854492s
----------------------------------------------
[INFO] 5152 files preprocessed! Took 0m 10s


In [33]:
# len(train)

In [34]:
# plt.figure(figsize=(1,1))
# plt.imshow(train[0][0], cmap='gray')
# plt.show

In [35]:
featureSet, labels = caer.sep_train(train, IMG_SIZE=img_size)

In [36]:
featureSet = caer.normalize(featureSet)
labels = to_categorical(labels, len(chars))

In [41]:
x_train, x_val, y_train, y_val = caer.train_val_split(featureSet, labels, val_ratio=1)
# x_train.shape
# y_train.shape

In [69]:
# Создаем модель нейронной сети
model = Sequential()

# Добавляем сверточный слой
model.add(Conv2D(64, (3, 3), activation='relu', input_shape=(180, 180, 1)))

# Добавляем слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# Добавляем второй сверточный слой
model.add(Conv2D(64, (3, 3), activation='relu'))

# Добавляем второй слой подвыборки
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))

# Преобразуем двумерные данные в одномерные
model.add(Flatten())

# Добавляем полносвязный слой
model.add(Dense(64, activation='relu'))

# Добавляем выходной слой
model.add(Dense(3, activation='softmax'))

optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.00001, decay=1e-7, momentum=0.7, nesterov=True)

# Компилируем модель
model.compile(optimizer, loss='binary_crossentropy', metrics=['accuracy'])
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_8 (Conv2D)           (None, 178, 178, 64)      640       
                                                                 
 max_pooling2d_8 (MaxPoolin  (None, 89, 89, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_9 (Conv2D)           (None, 87, 87, 64)        36928     
                                                                 
 max_pooling2d_9 (MaxPoolin  (None, 43, 43, 64)        0         
 g2D)                                                            
                                                                 
 flatten_4 (Flatten)         (None, 118336)            0         
                                                                 
 dense_6 (Dense)             (None, 64)               

In [54]:
del train
del featureSet
del labels
gc.collect()

NameError: name 'train' is not defined

In [70]:
BATCH_SIZE=10
EPOCHS=25

In [71]:
datagen = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, height_shift_range=0.1
                             , zoom_range=0.2)
train_gen = datagen.flow(x_train, y_train, batch_size=BATCH_SIZE)

In [72]:
callbacks_list=[LearningRateScheduler(canaro.lr_schedule)]

In [73]:
# optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=0.00001, decay=1e-7, momentum=0.7, nesterov=True)
# len(x_train)//10
len(y_val)//10

0

In [75]:
training=model.fit(train_gen, steps_per_epoch=64, epochs=EPOCHS,
                   validation_data=(x_val, y_val), validation_steps=8,
                   callbacks=callbacks_list)

Epoch 1/25
64/64 [==============================] - 13s 199ms/step - loss: 0.6293 - accuracy: 0.4000 - val_loss: 0.6916 - val_accuracy: 0.0000e+00 - lr: 0.0100
Epoch 2/25
64/64 [==============================] - 13s 195ms/step - loss: 0.6144 - accuracy: 0.4359 - lr: 0.0100
Epoch 3/25
64/64 [==============================] - 12s 194ms/step - loss: 0.6187 - accuracy: 0.4141 - lr: 0.0100
Epoch 4/25
64/64 [==============================] - 13s 207ms/step - loss: 0.5809 - accuracy: 0.5109 - lr: 0.0100
Epoch 5/25
64/64 [==============================] - 13s 204ms/step - loss: 0.5662 - accuracy: 0.5359 - lr: 0.0100
Epoch 6/25
64/64 [==============================] - 13s 199ms/step - loss: 0.5852 - accuracy: 0.5078 - lr: 0.0100
Epoch 7/25
64/64 [==============================] - 12s 194ms/step - loss: 0.5622 - accuracy: 0.5344 - lr: 0.0100
Epoch 8/25
64/64 [==============================] - 13s 196ms/step - loss: 0.5599 - accuracy: 0.5484 - lr: 0.0100
Epoch 9/25
64/64 [========================

In [78]:
val_datagen = canaro.generators.imageDataGenerator()
val_gen = val_datagen.flow(x_val, y_val, batch_size=BATCH_SIZE)
model.evaluate(val_gen)

1/1 [==============================] - 0s 224ms/step - loss: 0.3769 - accuracy: 1.0000


[0.37694883346557617, 1.0]

In [84]:
img1 = cv.imread('Data/kaggle_simpson_testset/kaggle_simpson_testset/homer_simpson_30.jpg')
def prepare(image, img_size):
    image = cv.cvtColor(image, cv.COLOR_RGB2GRAY)
    image = (image/10).astype('uint8')
    image = cv.resize(image, img_size)
    image = caer.reshape(image, img_size, 1)
    return image

In [86]:
img_size = (180,180)
predictions = model.predict(prepare(img1, img_size))

1/1 [==============================] - 0s 306ms/step


In [87]:
print(chars[np.argmax(predictions[0])])

homer_simpson
